# Runnung BIAFLOWS workflows locally

## Introduction

BIAFLOWS allows to package and run image analysis in a reproducible way. This notebook demonstrates how to run BIAFLOWS image analysis workflows on your own images on your local machine. 

You need to have docker installed and the docker-daemon running on your machine. 

## Available workflows

The two cells below will display a list of the available workflows. You can find out more about the workflows at:
* The [BIAFLOWS web-application](https://biaflows.neubias.org/) contains data-sets and allows to calculate benchmarks for the workflows.
* The workflows are hosted in the [neubiaswg5 organization on Dockerhub](https://hub.docker.com/search?q=neubiaswg5&type=image).
* All source code and configuration files are on [GitHub](https://github.com/orgs/Neubias-WG5/dashboard). 

In [50]:
%%capture biaflowsImages --no-stderr
!docker search --no-trunc neubiaswg5 | grep w_
lines = str(biaflowsImages).split("\r\n")

In [93]:
import pandas as pd
import numpy as np
workflows=[]
descriptions=[]
for line in lines:
    if line=='': 
        continue
    parts = line.split("  ")
    parts = list(filter(lambda a: a != '', parts))
    workflows.append(parts[0].split('/')[1])
    description = parts[1].lstrip()
    if (description=='0'):
        description = ''
    descriptions.append(description)

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
df = pd.DataFrame(zip(workflows,descriptions), columns=["workflow", "description"])
df.index = np.arange(1, len(df)+1)
display(df)

,workflow,description
1,w_spotdetection-icy,Neubias-WG5 benchmarking worfklow for spot detection using Icy
2,w_nucleisegmentation-imagej,"Segment clustered nuclei using a laplacian filter, thresholding and a binary watershed transform"
3,w_nucleisegmentation-cellprofiler,Automated build for CellProfiler workflow NucleiSegmentation.
4,w_spotdetection-ij,Docker image for the integration of the ImageJ spot-detection workflow into cytomine.
5,w_nucleisegmentation3d-imagej,"Segment 3D nuclei using Gaussian Blur 3D, thresholding, 3D fill holes and a binary Watershed 3D."
6,w_nucleisegmentation-maskrcnn,Nuclei segmentation using Mask-RCNN
7,w_nucleitracking-imagej,Track nuclei in a time series by using 3D object segmentation with watershed
8,w_filamenttracing3d-imagej,ImageJ macro to trace filaments
9,w_neurontracing_vaa3d,neuron tracing workflow using vaa3d
10,w_nucleisegmentation-ilastik,NEUBIAS WG5 nuclei segmentation workflow using ilastik 1.3.2 and Python post-processing.


## Running a workflow

To run a BIAFLOWS workflow you use the ``docker run`` command with the name of the image on dockerhub.
You will provide a folder under which input data is found and the output data is written. This folder must contain two subfolders:
* in - for the input images
* out - the workflow will write results into this folder

You can either choose an existing folder or execute the following cell to create the two folders.

In [95]:
!mkdir ./data
!mkdir ./data/in
!mkdir ./data/out

In this example we will execute the workflow ``w_nucleisegmentation-imagej`` for nuclei segementation in 2D images. If you do not have fitting input images, you can download example images by executing the following cell.

The workflow has two parameters:
* the radius for the Laplacian filter ``ij_radius``
* the segmentation threshold ``ij_threshold``
You might need to adapt the parameter values to your images.

The parameter of a workflow can be found in the [BIAFLOWS web-application](https://biaflows.neubias.org/) or in the [descriptor.json](https://github.com/Neubias-WG5/W_NucleiSegmentation-ImageJ/blob/master/descriptor.json) file of the workflow on github.

In [106]:
inFolder = "./data/in"
outFolder = "./data/out"
!docker run -v $(pwd)/data:/data -it neubiaswg5/w_nucleisegmentation-imagej:1.12.3 \
    --ij_radius 5 \
    --ij_threshold -0.5 \
    --infolder $(pwd)/data/in \
    --outfolder $(pwd)/data/out \
    -nmc \
    --local

Progress: 0  % ... Status: 2 - 'Initialisation...'
Progress: 25 % ... Status:   - 'Launching workflow...'
Progress: 90 % ... Status:   - 'Computing and uploading metrics...'
Progress: 100% ... Status: 3 - 'Finished.'
